In [1]:
from Pooling import Pooling_3D_density
from Conv_Layer import Conv_RBS_density_I2_3D
from load_dataset import load_MNIST, reduce_MNIST_dataset
import torch
import torch.nn as nn  # the neural network library of pytorch
from training import test_net, train_net

In [2]:
class QCNN(nn.Module):
    def __init__(self, I, O, J, K, device):
        super(QCNN, self).__init__()
        self.conv1 = Conv_RBS_density_I2_3D(I, K, J, device)
        self.pool1 = Pooling_3D_density(I, O, J, device)
        self.conv2 = Conv_RBS_density_I2_3D(O, K, J, device)
        self.pool2 = Pooling_3D_density(O, O // 2, J, device)
        self.fc = nn.Linear((O // 2) * (O // 2) * J, 10)

    def forward(self, x):
        c1 = self.conv1(x)
        p1 = self.pool1(c1)
        c2 = self.conv2(p1)
        p2 = self.pool2(c2)
        device_cpu = torch.device("cpu")
        d = torch.stack([torch.diag(p2[i]) for i in range(batch_size)]).to(device_cpu)
        output = self.fc(d)
        return output.to(device)  # return x for visualization


In [7]:


I = 16  # dimension of image we use
O = I // 2  # dimension after pooling
J = 4  # number of channel
K = 4  # size of kernel
k = 3
batch_size = 10  # batch number
scala = 6000  # time we reduce dataset
learning_rate = 1e-2
device = torch.device("mps")

train_loader, test_loader = load_MNIST(batch_size=batch_size)
reduced_loader = reduce_MNIST_dataset(train_loader, scala)
reduced_test_loader = reduce_MNIST_dataset(test_loader, 1000)

conv_network = QCNN(I, O, J, K, device)
optimizer = torch.optim.Adam(conv_network.parameters(), lr=learning_rate)
criterion = torch.nn.CrossEntropyLoss()

loss_list = []
accuracy_list = []
for epoch in range(1):
    train_loss, train_accuracy = train_net(batch_size, I, J, k, conv_network, train_loader, criterion, optimizer, device)
    loss_list.append(train_loss)
    accuracy_list.append(train_accuracy*100)
    print(f'Epoch {epoch}: Loss = {train_loss:.6f}, accuracy = {train_accuracy*100:.4f} %')

torch.save(conv_network.state_dict(), "model")

Epoch 0: Loss = 2.299336, accuracy = 10.0000 %


In [5]:
conv_network = QCNN(I, O, J, K, device)
conv_network.load_state_dict(torch.load("model"))
# conv_network.eval()
print("Model's state_dict:")
for param_tensor in conv_network.state_dict():
    print(param_tensor, "\t", conv_network.state_dict()[param_tensor].size())

Model's state_dict:
conv1.Parameters.0 	 torch.Size([])
conv1.Parameters.1 	 torch.Size([])
conv1.Parameters.2 	 torch.Size([])
conv1.Parameters.3 	 torch.Size([])
conv1.Parameters.4 	 torch.Size([])
conv1.Parameters.5 	 torch.Size([])
conv1.Parameters.6 	 torch.Size([])
conv1.Parameters.7 	 torch.Size([])
conv1.Parameters.8 	 torch.Size([])
conv1.Parameters.9 	 torch.Size([])
conv1.Parameters.10 	 torch.Size([])
conv1.Parameters.11 	 torch.Size([])
conv1.Parameters.12 	 torch.Size([])
conv1.Parameters.13 	 torch.Size([])
conv1.Parameters.14 	 torch.Size([])
conv1.Parameters.15 	 torch.Size([])
conv1.Parameters.16 	 torch.Size([])
conv1.Parameters.17 	 torch.Size([])
conv1.RBS_gates.0.angle 	 torch.Size([])
conv1.RBS_gates.1.angle 	 torch.Size([])
conv1.RBS_gates.2.angle 	 torch.Size([])
conv1.RBS_gates.3.angle 	 torch.Size([])
conv1.RBS_gates.4.angle 	 torch.Size([])
conv1.RBS_gates.5.angle 	 torch.Size([])
conv1.RBS_gates.6.angle 	 torch.Size([])
conv1.RBS_gates.7.angle 	 torch.Size(

In [8]:
test_loss, test_accuracy = test_net(batch_size, I, J, k, conv_network, reduced_test_loader, criterion, device)
print(f'Evaluation on test set: Loss = {test_loss:.6f}, accuracy = {test_accuracy*100:.4f} %')

Evaluation on test set: Loss = 2.294756, accuracy = 0.0000 %


In [9]:
total_params = sum(p.numel() for p in conv_network.parameters())
print(f"Number of parameters: {total_params}")

Number of parameters: 686
